In [1]:
import numpy as np
import pandas as pd
import os
import copy

In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [3]:
train_objs_num = len(train)
yTrain = train['revenue']
train.drop(['revenue'], axis=1, inplace=True)
trainData = pd.concat(objs=[train, test], axis=0)

In [4]:
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

In [5]:
def convertJsonToCol(name,data):
    new_dat = pd.DataFrame(data[name])
    new_dat[name] = new_dat[name].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
    dat = new_dat[name].str.get_dummies(',')
    return dat

In [6]:
genresTrain = convertJsonToCol('genres',trainData)
#genresTest = convertJsonToCol('genres',testData)

prodConTrain = convertJsonToCol('production_countries',trainData)
#prodConTest = convertJsonToCol('production_countries',testData)

prodComTrain = convertJsonToCol('production_companies',trainData)
#prodComTest = convertJsonToCol('production_companies',testData)

# spokenLanTrain = convertJsonToCol('spoken_languages',trainData)
# spokenLanTest = convertJsonToCol('spoken_languages',testData)

trainData = pd.concat([trainData, genresTrain], axis = 1, sort = False)
#testData = pd.concat([testData, genresTest], axis = 1, sort = False)
trainData = pd.concat([trainData, prodConTrain], axis = 1, sort = False)
#testData = pd.concat([testData, prodConTest], axis = 1, sort = False)
trainData = pd.concat([trainData, prodComTrain], axis = 1, sort = False)
#testData = pd.concat([testData, prodComTest], axis = 1, sort = False)
# trainData.head()
# trainData = pd.concat([trainData, spokenLanTrain], axis = 1, sort = False)
# testData = pd.concat([testData, spokenLanTest], axis = 1, sort = False)
trainData.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,Кинокомпания CTB,Кинокомпания «Lunapark»,Леополис,Наше Кино,Никола Фильм,Реал-Дакота,Роскинопрокат,СТВ,Флагман-Трейд,Фокс
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,0,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,0,0,0,0,0,0,0,0,0,0
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,0,0,0,0,0,0,0,0,0,0
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,0,0,0,0,0,0,0,0,0,0
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,0,0,0,0,0,0,0,0,0,0


In [7]:
trainData.drop(['genres','production_countries','production_companies','belongs_to_collection','homepage','imdb_id','poster_path','Keywords'], axis = 1, inplace = True)
#testData.drop(['genres','production_countries','production_companies','belongs_to_collection','homepage','imdb_id','poster_path','Keywords'], axis = 1, inplace = True)

In [8]:
def timeFeatures(df):
    df['datetime'] = pd.to_datetime(df['release_date'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["doy"]      = df["datetime"].dt.dayofyear
    df["year"]     = df['datetime'].dt.year
    df["month"]    = df['datetime'].dt.month
    df.drop(['datetime'], axis=1, inplace=True)
    return df

In [9]:
trainData = timeFeatures(trainData)
#testData = timeFeatures(testData)

In [10]:
trainData['title_word_count'] = np.where(trainData['title'].str.split().str.len()>=0,trainData['title'].str.split().str.len() ,0)
trainData['overview_word_count'] = np.where(trainData['overview'].str.split().str.len()>=0,trainData['overview'].str.split().str.len() ,0)
trainData['tagline_word_count'] = np.where(trainData['tagline'].str.split().str.len()>=0,trainData['tagline'].str.split().str.len() ,0)
trainData['original_title_letter_count'] = trainData['original_title'].str.len() 
trainData['original_title_word_count'] = trainData['original_title'].str.split().str.len() 

#testData['title_word_count'] = np.where(testData['title'].str.split().str.len()>=0,testData['title'].str.split().str.len() ,0)
#testData['overview_word_count'] = np.where(testData['overview'].str.split().str.len()>=0,testData['overview'].str.split().str.len() ,0)
#testData['tagline_word_count'] = np.where(testData['tagline'].str.split().str.len()>=0,testData['tagline'].str.split().str.len() ,0)
#testData['original_title_letter_count'] = testData['original_title'].str.len() 
#testData['original_title_word_count'] = testData['original_title'].str.split().str.len() 

In [11]:
trainData.drop(['title','overview','tagline','original_title','release_date'], axis = 1, inplace = True)
#testData.drop(['title','overview','tagline','original_title','release_date'], axis = 1, inplace = True)

In [12]:
new_dat = pd.DataFrame(trainData['spoken_languages'])
new_dat['spoken_languages'] = new_dat['spoken_languages'].map(lambda x: sorted([d['iso_639_1'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
dat = new_dat['spoken_languages'].str.get_dummies(',')
trainData = pd.concat([trainData, dat], axis = 1, sort = False)


#new_dat = pd.DataFrame(testData['spoken_languages'])
#new_dat['spoken_languages'] = new_dat['spoken_languages'].map(lambda x: sorted([d['iso_639_1'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
#dat = new_dat['spoken_languages'].str.get_dummies(',')
#testData = pd.concat([testData, dat], axis = 1, sort = False)

In [13]:
trainData.drop(['spoken_languages','cast','crew'],axis = 1, inplace = True)
#testData.drop(['spoken_languages','cast','crew'],axis = 1, inplace = True)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(list(trainData['original_language'].fillna('')))
trainData['original_language'] = le.transform(trainData['original_language'].fillna('').astype(str))
#testData['original_language'] = le.transform(testData['original_language'].fillna('').astype(str))

trainData.drop(['original_language','status'], axis=1, inplace=True)
#testData.drop(['original_language','status'], axis=1, inplace=True)
trainData.head()

,id,budget,popularity,runtime,Action,Adventure,Animation,Comedy,Crime,Documentary,...,ty,uk,ur,vi,wo,xh,xx,yi,zh,zu
0,1,14000000,6.575393,93,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,40000000,8.248895,113,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3300000,64.299990,105,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1200000,3.174936,122,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,1.148070,118,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
np.where(np.isnan(trainData))

(array([], dtype=int64), array([], dtype=int64))

In [16]:
train = copy.copy(trainData[:train_objs_num])
test = copy.copy(trainData[train_objs_num:])

In [26]:
#train.drop(['id'],axis=1, inplace=True)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=50)
model.fit(train,yTrain)
model.score(train,yTrain)

0.9534056615109451

In [18]:
#testData.drop(['overview_word_count','tagline_word_count'],axis=1,inplace=True)
#np.where(np.isnan(testData))

In [27]:
yTest = test['id']
test.drop(['id'],axis=1, inplace=True)

KeyError: 'id'

In [28]:
pred = model.predict(test)

In [29]:
#my_submission = pd.DataFrame({'id':yTest , 'revenue': pred})
yTest.id.head()
yTest = yTest.loc[:,~yTest.columns.duplicated()]
yTest.head()
#train.head()

,id
0,3001
1,3002
2,3003
3,3004
4,3005


In [22]:
yTrain.head()

0    12314651
1    95149435
2    13092000
3    16000000
4     3923970
Name: revenue, dtype: int64

In [30]:
my_submission = pd.DataFrame({'id':yTest['id'] , 'revenue': pred})

In [31]:
my_submission.to_csv('submission.csv', index=False)